In [ ]:
# direction is 0 = horizontal, 1 = vertical

gamma, alpha, beta, theta = 0.5, 0.5, 0.5, 0.5

class Particle:
    
    def __init__(self, road, direction):
        
        self.gamma = gamma
        
        self.position = [road.position, 0] if not road.direction else [0, road.position]
        self.direction = road.direction
        self.active = True
    
    def update_position(self):
        
        if not self.direction:
            self.position[0] += 1
        else:
            self.position[1] += 1
        
    def turn(self):
        
        self.direction = int(not self.direction)
        self.update_position()
    
    
class Road:
    
    def __init__(self, direction, position, dimension):
        
        self.alpha = alpha
        self.beta = beta
        
        self.grid = np.zeros((dimension,))
        self.particles = []
        self.direction = direction
        self.position = position
    
    def move_particle(self, particle):
    
        particle.update_position()
        self.update()
    
    def update(self):
        
        self.grid = np.zeros((dimension,))
        if not direction:
            for p in self.particles:
                self.grid[p.position[0]] = 1
                
        else:
            for p in self.particles:
                self.grid[p.position[1]] = 1
                
    def add_particle(self):
        
        new_particle = Particle(self, self.direction)
        self.particles.append(new_particle)
        self.update()
        
    def remove_particle(self, particle):
        
        self.particles.remove(particle)
        self.update()
        
    
class Intersection:
    
    def __init__(self, position):
        
        self.theta = theta
        
        self.position = position
        self.light = 0
        
    def change_light(self):
        
        self.light = int(not self.light)
    
    
class Environment:
    
    def __init__(self, num_horz_roads, num_vert_roads, dimension):
        
        horizontal_positions = np.random.choice(range(dimension), num_horz_roads, replace=False)
        vertical_positions = np.random.choice(range(dimension), num_vert_roads, replace=False)
        
        self.horz_roads = [Road(0, p, dimension) for p in horizontal_positions]
        self.vert_roads = [Road(1, p, dimension) for p in vertical_positions]
        
        grid = np.zeros((dimension, dimension))
        for road in self.horz_roads:
            grid[road.position, :] = road.grid
        for road in self.vert_roads:
            grid[:, road.position] = road.grid
            
        self.grid = grid
        
        self.intersections = []
        for y in self.horz_roads:
            for x in self.vert_roads:
                new_intersection = Intersection((x, y))
                self.intersections.append(new_intersection)
                
    def get_param(self):
        
        horz_param = sum([sum([p.gamma for p in road.particles]) for road in self.horz_roads])
        vert_param = sum([sum([p.gamma for p in road.particles]) for road in self.vert_roads])
        move_param = horz_param + vert_param
        add_param = sum([road.alpha for road in self.horz_roads + self.vert_roads])
        remove_param = sum([road.beta for road in self.horz_roads + self.vert_roads])
        light_param = sum([intersection.theta for intersection in self.intersections])
        
        return move_param + add_param + remove_param + light_param
    
    def choose_action(self):
        
        actions = ['move', 'add', 'remove', 'light']
        
        horz_param = sum([sum([p.gamma for p in road.particles]) for road in self.horz_roads])
        vert_param = sum([sum([p.gamma for p in road.particles]) for road in self.vert_roads])
        move_param = horz_param + vert_param
        add_param = sum([road.alpha for road in self.horz_roads + self.vert_roads])
        remove_param = sum([road.beta for road in self.horz_roads + self.vert_roads])
        light_param = sum([intersection.theta for intersection in self.intersections])
        
        probs = [move_param, add_param, remove_param, light_param]
        probs = probs / np.sum(probs)
        
        return np.random.choice(actions, p=probs)
        
    def move_particle(self):
        
        # choose road
        horz_p = [sum([p.gamma for p in road.particles]) for road in self.horz_roads]
        vert_p = [sum([p.gamma for p in road.particles]) for road in self.vert_roads]
        road = np.random.choice(self.horz_roads + self.vert_roads, p=horz_p + vert_p)
        
        # choose particle
        particle = np.random.choice(road.particles, p=[p.gamma for p in road.particles])
        
        # update
        road.move_particle(particle)
        
    def add_particle(self):
        
        roads = self.horz_roads + self.vert_roads
        probs = [road.alpha for road in roads]
        road = np.random.choice(roads, p=probs)
        road.add_particle()
        
    def remove_particle(self):
        
        roads = self.horz_roads + self.vert_roads
        probs = [road.beta for road in roads]
        road = np.random.choice(roads, p=probs)
        road.add_particle()
        
    def update_grid(self, road):
        
        if not road.direction:
            self.grid[road.position, :] = road.grid
        else:
            self.grid[:, road.position] = road.grid
        
        
class Logger:
    
    def __init__(self):
        
        self.log = []

    
class Simulation:
    
    def __init__(self):
        
        self.env = Environment(num_horz_roads, num_vert_roads, dimension)
        self.logger = Logger()
        self.clock = 0
        
    def visualize(self):
        
        clear_output(wait=True)
        print(self.grid)
    
    def take_action(self):
        
        param = self.env.get_param()
        time_lapse = np.random.exponential(scale=1/param)
        time.sleep(time_lapse)
        self.clock += time_lapse
        
        action = self.env.choose_action()
        
    def run(self):
        
        while self.clock < 60:
            self.take_action()
            self.visualize()

In [ ]:
sim = Simulation()
sim.run()